# Survival Analysis with Differential Privacy

## Data Preprocessing

Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from lifelines import KaplanMeierFitter

Read data

In [2]:
path_to_data = "../data/dataset/"

def load_data(name):
    data = pd.read_csv(os.path.join(path_to_data, name+".csv"))
    return data

Save results in a dictionary

In [3]:
names = ["cancer", "kidney", "leukemia", "mgus", "myeloid", "ovarian", "veteran"]
dfs = {name : load_data(name) for name in names}

## Laplacian Noise Time Obfuscation (LNTO)

@ARTICLE{Gondara_2019,title={Differentially Private Survival Function Estimation},year={2019},author={Lovedeep Gondara and Ke Wang}}

Survival function estimation is used in many disciplines, but it is most common in medical analytics in the form of the Kaplan-Meier estimator. Sensitive data (patient records) is used in the estimation without any explicit control on the information leakage, which is a significant privacy concern. We propose a first differentially private estimator of the survival function and show that it can be easily extended to provide differentially private confidence intervals and test statistics without spending any extra privacy budget. We further provide extensions for differentially private estimation of the competing risk cumulative incidence function. Using nine real-life clinical datasets, we provide empirical evidence that our proposed method provides good utility while simultaneously providing strong privacy guarantees.

In [ ]:
def LNTO(epsilon, df):
    return None